In [ ]:
'''
PositiveWords: 优 增长 崛起 提升 推进 回暖 涨停 发展 高速 增资 扩股 收购 领先 优势 标杆 升级 稳健 涨幅 利润 上市 收益 增加 稳定 提高 增值 盈利 增持
NegativeWords: 亏损 减少 下滑 新低 下跌 受贿 缺陷 下降 违约 虚假 误导性 亏损额 外债 减持 涉嫌 退市 债务 负债 违反 侵权
'''

# 实体抽取、分词

In [10]:
# https://github.com/lancopku/pkuseg-python/blob/master/example.txt ！！！！！！pkuseg 分词领域
# https://github.com/lancopku/pkuseg-python/blob/master/tags.txt
# https://blog.csdn.net/weixin_40411446/article/details/81014669  金融领域词典构建
# https://baijiahao.baidu.com/s?id=1633127045731574322&wfr=spider&for=pc 金融领域中的自然语言处理，弄懂这五个问题就够了 ！！！！！
#  https://openreview.net/forum?id=ry018WZAZ 这片文章把active learning应用到了CNN-CNN-LSTM模型，用于处理NER问题，也就是seq labeling问题。它能够仅使用25%的数据，达到state-of-the-art的水平

# 把搜狗输入法金融词库转成txt就行了
# baidu 金融领域 分词
# https://zhuanlan.zhihu.com/p/32269563 重磅课程：中文自然语言理解在金融领域的应用
# http://manu44.magtech.com.cn/Jwk_infotech_wk3/article/2018/2096-3467/2096-3467-2-10-95.shtml  基于深度学习的领域情感词典自动构建
# https://www.docin.com/p-2179797554.html
#  http://tools.bugscaner.com/sceltotxt/  搜狗词库在线转换scel 为 txt
'''
处理流程：
1、预先设置部分字典信息（通过搜狗词库字典初始化一批）
2、通过pkuseg 分词及词性，组合新的命名实体， 更新到字典信息中去
3、后期可采用 bert ， 根据字典信息生成标注信息
'''
import pkuseg
file_name = 'D:\\PROJECT_TW\\git\\finance\\data\\analysis\\news_1.txt'
lexicon_file = 'D:\\PROJECT_TW\\git\\finance\\data\\dict.txt'
seg = pkuseg.pkuseg(model_name='news', postag=True, user_dict=lexicon_file)
with open(file_name, 'r', encoding='utf-8') as f:
    nlines = f.readlines()
tokens = [] 
# for l in nlines:
#    tokens.extend(seg.cut(l))
text = '''“随着国产替代以及自主可控的战略要求，拓维信息股份有限公司可以预见鲲鹏服务器未来将在更多关乎国家信息安全
         的应用场景中得以启用。”携手华为共建鲲鹏产业生态, 长期负债合计'''
print(seg.cut(text))



[('“', 'w'), ('随着', 'p'), ('国产', 'b'), ('替代', 'vn'), ('以及', 'c'), ('自主', 'vd'), ('可控', 'v'), ('的', 'u'), ('战略', 'n'), ('要求', 'n'), ('，', 'w'), ('拓维信息', 'nso'), ('股份有限公司', 'n'), ('可以', 'v'), ('预见', 'v'), ('鲲鹏', 'nz'), ('服务器', 'n'), ('未来', 't'), ('将', 'd'), ('在', 'p'), ('更', 'd'), ('多', 'a'), ('关乎', 'v'), ('国家', 'n'), ('信息', 'n'), ('安全', 'an'), ('的', 'u'), ('应用', 'vn'), ('场景', 'n'), ('中', 'f'), ('得以', 'v'), ('启用', 'v'), ('。', 'w'), ('”', 'w'), ('携手', 'v'), ('华为', 'nz'), ('共建', 'v'), ('鲲鹏', 'n'), ('产业', 'n'), ('生态', 'n'), (',', 'n'), ('长期', 'd'), ('负债', 'v'), ('合计', 'v')]


In [7]:
# 处理搜狗转换过来的
file_name = u'D:\\PROJECT_TW\\git\\finance\\data\\sogou\\沪深股票名称.txt'
out_file_name = u'D:\\PROJECT_TW\\git\\finance\\data\\sogou\\沪深股票名称_ext.txt'
ftype = 'nso'
with open(file_name, 'r', encoding='utf-8') as f:
    flines = f.readlines()

with  open(out_file_name, 'w', encoding='utf-8') as f:
    f.writelines(['{}\t{}\n'.format(x.strip(), ftype) for x in flines])

# 实体关系、属性

In [ ]:
# https://zhuanlan.zhihu.com/p/91762831?utm_source=wechat_session&utm_medium=social&utm_oi=777186450763440128
# https://towardsdatascience.com/embedding-models-for-knowledge-graph-completion-a66d4c01d588   !!!!!!!!!!!
# https://zhuanlan.zhihu.com/p/34638021 Entity resolution相关
# https://blog.csdn.net/Matt_sh/article/details/106225479 关系抽取比赛：pipeline方法实践
# 实体的关系的抽取方法可以简单分为两类：一类是pipeline抽取方法。另一类是并行或联合抽取方法。
# https://zhuanlan.zhihu.com/p/39205829 知识图谱入门 (三) 知识抽取  !!!!
# https://blog.csdn.net/lt326030434/article/details/88058739  实体消岐
# https://blog.csdn.net/u012736685/article/details/97371530 实体消岐
# https://blog.csdn.net/qq_27590277/article/details/107133347?%3E nlp中的实体关系抽取方法总结   !!!!!!!!!
# https://github.com/yuanxiaosc/Entity-Relation-Extraction !!!!!!!!!!!!!!!!!!!!!!1
# https://arxiv.org/abs/1905.08284  这篇文章从学术角度，没有多大的贡献，只是发现了一种标注实体的方式，十分契合bert
# https://zhuanlan.zhihu.com/p/77868938 nlp中的实体关系抽取方法总结